In [2]:
import numpy as np
import pandas as pd
import string
import re
from IPython.core.display import HTML

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
import stanza

# cambiador de formatos para factorizar un dataset
from sklearn.feature_extraction.text import TfidfVectorizer 



# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

c:\Users\Sheen\anaconda3\envs\Samsung_Campus\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from nltk.corpus import stopwords
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [4]:
from nltk.corpus import stopwords
spanish_stopwords = list(stopwords.words('spanish'))
spanish_stopwords.extend(("xq", "oye", "dale", "dele", "ba", "abc", "nan", "na", "todo bien"))
print(spanish_stopwords)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [5]:
ruta = 'Opiniones.csv'
data = pd.read_csv(ruta)
# print(data.head())

In [6]:
data.rename(columns={'AÑO':'YEAR'}, inplace=True)
print(data.head())

     YEAR    SEDE    FACULTAD          CARRERA OBSERVACIONES
0  2019.0  SEDE 2  FACULTAD 1  LICENCIATURA 03             0
1  2019.0  SEDE 2  FACULTAD 1  LICENCIATURA 04             0
2  2019.0  SEDE 2  FACULTAD 2  LICENCIATURA 07             0
3  2019.0  SEDE 2  FACULTAD 3    INGENIERIA 15             0
4  2019.0  SEDE 3  FACULTAD 1  LICENCIATURA 04             0


In [7]:
def limpiar_tokenizar(texto):
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', texto)
    # Se convierte todo el texto a minúsculas
    nuevo_texto = nuevo_texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\
    \-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~—]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]
    # quitar los token que sean solamente de una letra repetidas 2 o mas veces
    nuevo_texto = [token for token in nuevo_texto if not re.fullmatch(r'(.)\1*', token)]
    # une los tokens en un solo string
    nuevo_texto = ' '.join(nuevo_texto)
    
    return(nuevo_texto)

test = "Esto es 1 ejemplo de l'limpieza de6 TEXTO  https://t.co/rnHPgyhx4Z @cienciadedatos #textmining zzzzz oooooo aaaaaaaa hhh eeeeeeee todo bien, bien"
print(test)
print(limpiar_tokenizar(texto=test))

Esto es 1 ejemplo de l'limpieza de6 TEXTO  https://t.co/rnHPgyhx4Z @cienciadedatos #textmining zzzzz oooooo aaaaaaaa hhh eeeeeeee todo bien, bien
esto es ejemplo de limpieza de texto cienciadedatos textmining todo bien bien


In [8]:
print(data['FACULTAD'].value_counts())
print("______________________________________________________________________")
print(string.punctuation)
print(re.escape(string.punctuation))
print("______________________________________________________________________")
data['OBSERVACIONES'] = data['OBSERVACIONES'].astype(str)
data['TEXTO_TOKEN'] = data['OBSERVACIONES'].apply(lambda x: limpiar_tokenizar(x))
# Selecciona una cantidad aleatoria de filas
random_data = data.sample(20)


# Muestra las filas seleccionadas como una tabla HTML
display(HTML(random_data.to_html()))


FACULTAD
FACULTAD 3    13765
FACULTAD 1    12296
FACULTAD 5    11577
FACULTAD 4     7266
FACULTAD 2     5828
FACULTAD 6     1787
Name: count, dtype: int64
______________________________________________________________________
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}\~
______________________________________________________________________


,YEAR,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN
30039,2019.0,SEDE 1,FACULTAD 5,LICENCIATURA 22,Mejor atención en la cafeteria,mejor atención en la cafeteria
27767,2019.0,SEDE 6,FACULTAD 4,LICENCIATURA 14,Más estacionamientos!,más estacionamientos
49044,2019.0,SEDE 1,FACULTAD 5,POSTGRADO 34,Tecnología Educativa Avanzada,tecnología educativa avanzada
6469,2019.0,SEDE 1,FACULTAD 5,LICENCIATURA 23,Algunos precios de las maquinas expendedoras son muy elevados,algunos precios de las maquinas expendedoras son muy elevados
12513,2019.0,SEDE 1,FACULTAD 6,LICENCIATURA 24,"Disponibilidad de atencion universitaria (cajas, atencion medica, secretaria academuca, fotocopias, serviciow informativos) total ausencia en turno nocturno",disponibilidad de atencion universitaria cajas atencion medica secretaria academuca fotocopias serviciow informativos total ausencia en turno nocturno
29940,2019.0,SEDE 6,FACULTAD 4,POSTGRADO 33,Mejor acondicionamiento de laboratorios.,mejor acondicionamiento de laboratorios
22817,2019.0,SEDE 1,FACULTAD 1,INGENIERIA 01,La eficiencia en los centros de computo,la eficiencia en los centros de computo
49191,2019.0,SEDE 1,FACULTAD 3,LICENCIATURA 09,Tener más centros de fotocopiados.,tener más centros de fotocopiados
37112,2019.0,SEDE 6,FACULTAD 3,POSTGRADO 24,Metedologia,metedologia
2760,2019.0,SEDE 1,FACULTAD 5,POSTGRADO 26,.,


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52583 entries, 0 to 52582
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   YEAR           52519 non-null  float64
 1   SEDE           52519 non-null  object 
 2   FACULTAD       52519 non-null  object 
 3   CARRERA        52338 non-null  object 
 4   OBSERVACIONES  52583 non-null  object 
 5   TEXTO_TOKEN    52583 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.4+ MB


In [10]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52338 entries, 0 to 52518
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   YEAR           52338 non-null  float64
 1   SEDE           52338 non-null  object 
 2   FACULTAD       52338 non-null  object 
 3   CARRERA        52338 non-null  object 
 4   OBSERVACIONES  52338 non-null  object 
 5   TEXTO_TOKEN    52338 non-null  object 
dtypes: float64(1), object(5)
memory usage: 2.8+ MB


In [11]:
data['YEAR'] = data['YEAR'].astype(float).astype(int)
col_list = ['YEAR','SEDE', 'FACULTAD', 'CARRERA']
for col in col_list:
    if col == 'TEXTO_TOKEN':
        data[col] = data[col].apply(lambda x: ' '.join(x))
    print(f'Columna {col}: {data[col].nunique()} subniveles')

Columna YEAR: 1 subniveles
Columna SEDE: 8 subniveles
Columna FACULTAD: 6 subniveles
Columna CARRERA: 103 subniveles


In [12]:
data.drop(columns='YEAR', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52338 entries, 0 to 52518
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SEDE           52338 non-null  object
 1   FACULTAD       52338 non-null  object
 2   CARRERA        52338 non-null  object
 3   OBSERVACIONES  52338 non-null  object
 4   TEXTO_TOKEN    52338 non-null  object
dtypes: object(5)
memory usage: 2.4+ MB


In [13]:
display(HTML(data.sample(10).to_html()))

,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN
165,SEDE 1,FACULTAD 5,POSTGRADO 10,1,
41948,SEDE 1,FACULTAD 2,INGENIERIA 08,No tengo,no tengo
36242,SEDE 1,FACULTAD 1,INGENIERIA 01,Mejorar wifi para estudiantes,mejorar wifi para estudiantes
3173,SEDE 1,FACULTAD 3,INGENIERIA 15,.,
16635,SEDE 8,FACULTAD 3,INGENIERIA 15,Escases de estacionamiento cuando la universidad cuenta con mucho más áreas verdes que de edi----aciones.,escases de estacionamiento cuando la universidad cuenta con mucho más áreas verdes que de edi aciones
6245,SEDE 1,FACULTAD 3,POSTGRADO 22,Al hacer los pagos presenciales hay demasiado desorden y muchas veces toca formar filas larguísimas 2 veces,al hacer los pagos presenciales hay demasiado desorden muchas veces toca formar filas larguísimas veces
6518,SEDE 1,FACULTAD 1,INGENIERIA 01,"Algunos pupitres, mesas, y bancos del edificio #4 no están en un estado apropiado.",algunos pupitres mesas bancos del edificio no están en un estado apropiado
33591,SEDE 6,FACULTAD 2,TÉCNICO 05,Mejorar instalaciones,mejorar instalaciones
2505,SEDE 1,FACULTAD 5,LICENCIATURA 22,.,
46665,SEDE 3,FACULTAD 6,POSTGRADO 12,Satisfecho,satisfecho


In [14]:
data = data[data['TEXTO_TOKEN'].apply(len) > 0]
print(f'Tamaño del set después de eliminar las filas sin tokens: {data.shape}')
display(HTML(data.head(10).to_html()))

Tamaño del set después de eliminar las filas sin tokens: (46376, 5)


,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN
878,SEDE 4,FACULTAD 3,LICENCIATURA 10,"- Capacidad, iluminación, ventilación de las aulas. El aire se suele dañar Los salones se ven muy oscuros, en mi opinión no motivan a dar clase ahí. - Servicios de Fotocopiado. Se necesita urgente.",capacidad iluminación ventilación de las aulas el aire se suele dañar los salones se ven muy oscuros en mi opinión no motivan dar clase ahí servicios de fotocopiado se necesita urgente
879,SEDE 8,FACULTAD 4,LICENCIATURA 15,---- de reciclado,de reciclado
880,SEDE 1,FACULTAD 5,INGENIERIA 25,deberían modernizar los equipos utilizados en los laboratorios.,deberían modernizar los equipos utilizados en los laboratorios
881,SEDE 1,FACULTAD 4,INGENIERIA 17,"---- en necesidad del remodelación interior, abrir mas entradas de estacionamientos (hay una que se construyo pero esta cerrada), mas puentes peatonales y reparar los artefactos de los baños.",en necesidad del remodelación interior abrir mas entradas de estacionamientos hay una que se construyo pero esta cerrada mas puentes peatonales reparar los artefactos de los baños
882,SEDE 1,FACULTAD 1,INGENIERIA 01,---- necesita cortinas para uso de proyectores,necesita cortinas para uso de proyectores
883,SEDE 8,FACULTAD 1,INGENIERIA 01,no contamos con servicio de fotocopia. El Wi-Fi no llega a ciertas aulas en donde muchas veces se necesita para trabajos que nos asignan.,no contamos con servicio de fotocopia el wi fi no llega ciertas aulas en donde muchas veces se necesita para trabajos que nos asignan
884,SEDE 1,FACULTAD 5,LICENCIATURA 27,---- no tiene un buen olor,no tiene un buen olor
885,SEDE 8,FACULTAD 3,INGENIERIA 15,"WiFi no es suficiente, implementar un WiFi con mayor capacidad de banda ancha y mayores velocidades sería oportuno.",wifi no es suficiente implementar un wifi con mayor capacidad de banda ancha mayores velocidades sería oportuno
893,SEDE 1,FACULTAD 2,TÉCNICO 05,#14 / pocas charlas o seminarios de innovación de información enfocado para el grupo nocturno,pocas charlas seminarios de innovación de información enfocado para el grupo nocturno
894,SEDE 1,FACULTAD 2,POSTGRADO 20,#14 Informar sobre Servicios de Orientación para el horario estudiantil nocturno.,informar sobre servicios de orientación para el horario estudiantil nocturno


In [15]:
dato_sample= data.sample(100)

In [16]:
nlp = stanza.Pipeline(lang='es')

2024-06-13 12:43:24 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-06-13 12:43:25 INFO: Downloaded file to C:\Users\Sheen\stanza_resources\resources.json
2024-06-13 12:43:27 INFO: Loading these models for language: es (Spanish):
| Processor    | Package         |
----------------------------------
| tokenize     | ancora          |
| mwt          | ancora          |
| pos          | ancora_charlm   |
| lemma        | ancora_nocharlm |
| constituency | combined_charlm |
| depparse     | ancora_charlm   |
| sentiment    | tass2020_charlm |
| ner          | conll02         |

2024-06-13 12:43:27 INFO: Using device: cpu
2024-06-13 12:43:27 INFO: Loading: tokenize
2024-06-13 12:43:27 INFO: Loading: mwt
2024-06-13 12:43:27 INFO: Loading: pos
2024-06-13 12:43:27 INFO: Loading: lemma
2024-06-13 12:43:27 INFO: Loading: constituency
2024-06-13 12

In [17]:
display(HTML(dato_sample.head(20).to_html()))

,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN
20556,SEDE 1,FACULTAD 5,LICENCIATURA 21,Incluir más materias en la modalidad virtual,incluir más materias en la modalidad virtual
48920,SEDE 1,FACULTAD 3,INGENIERIA 15,Tal vez poner cámaras que apunten hacia las paradas,tal vez poner cámaras que apunten hacia las paradas
18123,SEDE 1,FACULTAD 3,LICENCIATURA 12,extender horarios de atencion de secretarias,extender horarios de atencion de secretarias
41227,SEDE 1,FACULTAD 4,TÉCNICO 06,No hay papel en los baños,no hay papel en los baños
38266,SEDE 1,FACULTAD 5,LICENCIATURA 27,nada,nada
36073,SEDE 1,FACULTAD 5,INGENIERIA 25,Mejorar sistema de matrícula,mejorar sistema de matrícula
24432,SEDE 1,FACULTAD 3,LICENCIATURA 10,Las materias deberían incluir inglés,las materias deberían incluir inglés
35583,SEDE 3,FACULTAD 2,TÉCNICO 05,Mejorar los horarios,mejorar los horarios
49835,SEDE 1,FACULTAD 2,INGENIERIA 08,todo bien,todo bien
35345,SEDE 7,FACULTAD 3,LICENCIATURA 08,Mejorar limpieza de los baños,mejorar limpieza de los baños


https://colab.research.google.com/drive/1zw26doucuZr1cSUZPQ1kjnVA8NuxRcne#scrollTo=-fofZJL0eHwZ

In [18]:
excluded_words = ['cual', 'que', 'se', 'nos', 'todo', 'el', 'la', 'las', 'nadie', 'nada', 'falta']

In [19]:
def obtener_dependencias(texto):    
    for sent in texto.sentences:
        for dep in sent.dependencies:
            if (dep[1] == 'obj' or dep[1] == 'nsubj') and dep[2].text not in excluded_words:
                print(dep[2].text)
                return dep[2].text

In [20]:
dato_sample['dependencias'] = dato_sample['TEXTO_TOKEN'].apply(lambda x: obtener_dependencias(nlp(x)))

materias
cámaras
horarios
papel
sistema
materias
horarios
limpieza
tipo
estacionamientos
calendarios
profesores
cajeros
aires
cafetería
aulas
wifi
salones
alguien
baños
sistema
espacios
maquina
form
capacidad
observaciones
tips
salones
mejoramientos
internet
wifi
internet
salón
laboratorios
horario
proyectores
internet
materiales
mesas
opciones
página
laboratorios
copiadoras
le
cafetería
wifi
administrativo
baños
atencion
instrumentación
distancia


In [21]:
def obtener_nucleos(texto):
    for sent in texto.sentences:
        for word in sent.words:
            if word.deprel == 'nsubj' and word.text not in excluded_words:
                print(word.text)
                return word.text

In [22]:
dato_sample['nucleo'] = dato_sample['TEXTO_TOKEN'].apply(lambda x: obtener_nucleos(nlp(x)))

materias
tipo
profesores
aires
cafetería
baños
sistema
biblioteca
maquina
señal
ninguno
internet
salón
proyectores
página
laboratorios
estudiantes
cafetería
wifi
baños


In [23]:
display(HTML(dato_sample.to_html()))

,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN,dependencias,nucleo
20556,SEDE 1,FACULTAD 5,LICENCIATURA 21,Incluir más materias en la modalidad virtual,incluir más materias en la modalidad virtual,materias,None
48920,SEDE 1,FACULTAD 3,INGENIERIA 15,Tal vez poner cámaras que apunten hacia las paradas,tal vez poner cámaras que apunten hacia las paradas,cámaras,None
18123,SEDE 1,FACULTAD 3,LICENCIATURA 12,extender horarios de atencion de secretarias,extender horarios de atencion de secretarias,horarios,None
41227,SEDE 1,FACULTAD 4,TÉCNICO 06,No hay papel en los baños,no hay papel en los baños,papel,None
38266,SEDE 1,FACULTAD 5,LICENCIATURA 27,nada,nada,None,None
36073,SEDE 1,FACULTAD 5,INGENIERIA 25,Mejorar sistema de matrícula,mejorar sistema de matrícula,sistema,None
24432,SEDE 1,FACULTAD 3,LICENCIATURA 10,Las materias deberían incluir inglés,las materias deberían incluir inglés,materias,materias
35583,SEDE 3,FACULTAD 2,TÉCNICO 05,Mejorar los horarios,mejorar los horarios,horarios,None
49835,SEDE 1,FACULTAD 2,INGENIERIA 08,todo bien,todo bien,None,None
35345,SEDE 7,FACULTAD 3,LICENCIATURA 08,Mejorar limpieza de los baños,mejorar limpieza de los baños,limpieza,None
